# Radial Profile

This example shows how to create a radial profile from a SOXS event file, including using an exposure map to get flux-based quantities. We'll simulate a simple isothermal cluster.

In [ ]:
import matplotlib
matplotlib.rc("font", size=18)
import matplotlib.pyplot as plt
import soxs
import astropy.io.fits as pyfits

First, create the spectrum for the cluster using an absorbed thermal APEC model:

In [ ]:
emin = 0.05 # keV
emax = 20.0 # keV
nbins = 20000 
agen = soxs.ApecGenerator(emin, emax, nbins)

In [ ]:
kT = 6.0
abund = 0.3
redshift = 0.05
norm = 1.0
spec = agen.get_spectrum(kT, abund, redshift, norm)
spec.rescale_flux(1.0e-13, emin=0.5, emax=2.0, flux_type="energy")
spec.apply_foreground_absorption(0.02)

And a spatial distribution based on a $\beta$-model:

In [ ]:
pos = soxs.BetaModel(30.0, 45.0, 50.0, 0.67)

Generate a SIMPUT catalog from these two models, and write it to a file:

In [ ]:
width = 10.0 # arcmin by default
nx = 1024 # resolution of image
cluster = soxs.SimputSpectrum.from_models("beta_model", spec, pos, width, nx)
cluster_cat = soxs.SimputCatalog.from_source("beta_model.simput", cluster, overwrite=True)

and run the instrument simulation (for simplicity we'll turn off the point-source background):

In [ ]:
soxs.instrument_simulator("beta_model.simput", "evt.fits", (100.0, "ks"), "lynx_hdxi", [30., 45.], 
                          overwrite=True, ptsrc_bkgnd=False)

Make an exposure map so that we can obtain flux-based quantities:

In [ ]:
soxs.make_exposure_map("evt.fits", "expmap.fits", 2.3, overwrite=True)

Make the radial profile, using energies between 0.5 and 5.0 keV, between radii of 0 and 200 arcseconds, with 50 bins:

In [ ]:
soxs.write_radial_profile("evt.fits", "profile.fits", [30.0, 45.0], 
                          0, 200, 50, emin=0.5, emax=5.0, expmap_file="expmap.fits", overwrite=True)

Now we can use AstroPy's FITS reader to open the profile and have a look at the columns that are inside:

In [ ]:
f = pyfits.open("profile.fits")
f["PROFILE"].columns

and use Matplotlib to plot some quantities. We can plot the surface brightness:

In [ ]:
plt.figure(figsize=(8,8))
plt.errorbar(f["profile"].data["rmid"], f["profile"].data["sur_bri"], lw=2, yerr=f["profile"].data["sur_bri_err"])
plt.xscale('log')
plt.yscale('log')
plt.xlabel("r (arcsec)")
plt.ylabel("S (cts/s/arcsec**2)")

and, since we used an exposure map, the surface flux:

In [ ]:
plt.figure(figsize=(8,8))
plt.errorbar(f["profile"].data["rmid"], f["profile"].data["sur_flux"], lw=2, yerr=f["profile"].data["sur_flux_err"])
plt.xscale('log')
plt.yscale('log')
plt.xlabel("r (arcsec)")
plt.ylabel("S (cts/s/cm**2/arcsec**2)")